In [1]:
import numpy as np
import awkward as ak
import uproot
import src.functions as fn

In [2]:
filename = "ntuples_13723003_0.root"
file = uproot.open(filename)
tracksters = file["ticlNtuplizer/tracksters"]
vertices_indexes = tracksters['vertices_indexes'].array()
vertices_x = tracksters['vertices_x'].array()
vertices_y = tracksters['vertices_y'].array()
vertices_z = tracksters['vertices_z'].array()
vertices_E = tracksters['vertices_energy'].array()

In [3]:
vertices_indexes_00 = vertices_indexes[0,0]
vertices_x_00 = vertices_x[0,0]
vertices_y_00 = vertices_y[0,0]
vertices_z_00 = vertices_z[0,0]
vertices_E_00 = vertices_E[0,0]

In [4]:
vertices_indexes_0 = vertices_indexes[0]
vertices_x_0 = vertices_x[0]
vertices_y_0 = vertices_y[0]
vertices_z_0 = vertices_z[0]
vertices_E_0 = vertices_E[0]

In [5]:
indexes = np.stack([ak.to_numpy(vertices_indexes_00)]*len(vertices_indexes_00),axis=0)
enMtrx = np.stack([ak.to_numpy(vertices_E_00)]*len(vertices_E_00),axis=0)
energyMask = enMtrx > np.transpose(enMtrx)
distMatr = fn.euclideanMatrix(vertices_x_00, vertices_y_00, vertices_z_00)
distSort = np.argsort(distMatr,axis=1)
#distMatrSorted = distMatr[np.arange(distMatr.shape[0])[:,None], distSort]
energyMaskSorted = energyMask[np.arange(energyMask.shape[0])[:,None], distSort]
indexesSorted = indexes[np.arange(energyMask.shape[0])[:,None], distSort]
indexesSorted = ak.unflatten(ak.flatten(indexesSorted), ak.num(indexesSorted))
energyMaskSorted = ak.unflatten(ak.flatten(energyMaskSorted), ak.num(energyMaskSorted))
indexesSorted = indexesSorted[energyMaskSorted]
indexesSorted = indexesSorted[:,:5]
ak.cartesian([indexesSorted, vertices_indexes_00])

<Array [[(9, 1), (26, 1, ... 261), (134, 261)]] type='27 * var * (int64, int64)'>

In [8]:
def edgeBuilderNew(vertices_indexes, vertices_x, vertices_y, vertices_z, vertices_E, nEdg=1):
    if len(vertices_indexes) <= nEdg:
        raise ValueError("Number of attempted connections 'nEdg' cannot exceed the size of the graph")
    # Create matrix of indexes
    indexes = np.stack([ak.to_numpy(vertices_indexes)]*len(vertices_indexes),axis=0)
    # Perform energy filtering. Keep only nodes that have greater energy that the specified node
    enMtrx = np.stack([ak.to_numpy(vertices_E)]*len(vertices_E),axis=0)
    energyMask = enMtrx > np.transpose(enMtrx)
    # Calculate euclidean distance between all nodes and apply energy mask
    distMatr = fn.euclideanMatrix(vertices_x, vertices_y, vertices_z)
    # Sort each row and keep the indexes of the sorted arrays
    distSort = np.argsort(distMatr,axis=1)
    # Sort the euclidean distance using the indexes from previous step
    energyMaskSorted = energyMask[np.arange(energyMask.shape[0])[:,None], distSort]
    indexesSorted = indexes[np.arange(energyMask.shape[0])[:,None], distSort]
    # Some awkward magic - converts innermost array length from const to var
    indexesSorted = ak.unflatten(ak.flatten(indexesSorted), ak.num(indexesSorted))
    energyMaskSorted = ak.unflatten(ak.flatten(energyMaskSorted), ak.num(energyMaskSorted))
    # Filter nodes that have lower energy and keep nEdg nearest neighbors
    indexesSorted = indexesSorted[energyMaskSorted]
    indexesSorted = indexesSorted[:,:nEdg]
    return ak.cartesian([vertices_indexes, indexesSorted])

In [19]:
%%timeit
edgeBuilderNew(vertices_indexes_00, vertices_x_00, vertices_y_00, vertices_z_00, vertices_E_00, nEdg=5)

5.35 ms ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
fn.edgeBuilderNNTr(vertices_indexes_0, vertices_x_0, vertices_y_0, vertices_z_0, vertices_E_0)

467 ms ± 4.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
